In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import glob
import os
import datetime
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_datasets

ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


# For TRENDY SAT region

## Calculate the mean of a variable including only specific model categories

In [ ]:
# NEW model categories
models_very_good_list = ['CLASSIC', 'OCN']
models_very_good_list_for_fFire = ['CLASSIC']
models_ok_list = ['ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE']
models_ok_list_for_fFire = ['ISBA-CTRIP']
models_bad_low_amplitude_new = ['ORCHIDEE-CNP', 'LPJ', 'JULES-ES-1p0', 'JSBACH', 'IBIS']
models_bad_low_amplitude_new_for_ra_rh_mean = ['LPJ', 'JSBACH', 'IBIS'] # all models containing respiration & without ORCHIDEE-CNP because it is very bad for respiration
models_bad_low_amplitude_new_for_ra_rh_fFire = ['LPJ', 'JSBACH']
models_bad_high_amplitude_new = ['SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
models_bad_high_amplitude_new_for_npp_mean = ['SDGVM', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3'] # all models except VISIT because this models does not have npp
models_bad_high_amplitude_new_for_fFire = ['SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0']

'''# WRONG: NEW model categories
models_very_good_list = ['ORCHIDEE', 'OCN']
models_ok_list = ['ISBA-CTRIP', 'ISAM', 'CLASSIC', 'YIBs']
models_bad_low_amplitude_new = ['ORCHIDEE-CNP', 'LPJ', 'JULES-ES-1p0', 'JSBACH', 'IBIS']
models_bad_low_amplitude_new_for_ra_rh_mean = ['LPJ', 'JSBACH', 'IBIS'] # all models containing respiration & without ORCHIDEE-CNP because it is very bad for respiration
models_bad_high_amplitude_new = ['SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
models_bad_high_amplitude_new_for_npp_mean = ['SDGVM', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3'] # all models except VISIT because this models does not have npp
'''
Variables = ['gpp', 'nbp', 'npp', 'ra', 'rh', 'fFire']
Variables_new = ['gpp', 'nbp', 'npp', 'ra', 'rh', 'ra+rh', 'ra+rh-gpp', 'nbp-(ra+rh)+gpp']
#Variable = ['fFire', 'ra+rh-gpp+fFire']
Variable = ['nbp-(ra+rh-gpp+fFire)'] # works only for vg & ok models

# generate files for ensemble mean:
for var in Variable:
    column_name_list_very_good = []
    column_name_list_ok = []
    column_name_list_bad_low = []
    column_name_list_bad_high = []
    
    if var == 'ra+rh':
        variable = 'ratot+rh'
    elif var=='ra+rh-gpp':
        variable = 'ratot+rhtot-gpp'
    elif var=='nbp-(ra+rh)+gpp':
        variable = 'nbptot-(ratot+rhtot)+gpp'
    elif var=='ra+rh-gpp+fFire':
        variable = 'ratot+rhtot-gpptot+fFire'
    elif var=='nbp-(ra+rh-gpp+fFire)':
        variable = 'nbptot-(ratot+rhtot-gpptot+fFiretot)'
    else:
        variable = var
    
    if var=='fFire' or var=='ra+rh-gpp+fFire':
        for model in models_very_good_list:
            column_name_list_very_good.append(variable + 'tot_TgC_month_' + model)
    elif var=='nbp-(ra+rh-gpp+fFire)':
        for model in models_very_good_list:
            column_name_list_very_good.append(variable + '_TgC_month_' + model)
    else:
        for model in models_very_good_list:
            column_name_list_very_good.append(variable + 'tot_TgC_month_' + model)
    
    if var=='fFire' or var=='ra+rh-gpp+fFire':
        for model in models_ok_list:
            column_name_list_ok.append(variable + 'tot_TgC_month_' + model)
    elif var=='nbp-(ra+rh-gpp+fFire)':
        for model in models_ok_list:
            column_name_list_ok.append(variable + '_TgC_month_' + model)
    else:
        for model in models_ok_list:
            column_name_list_ok.append(variable + 'tot_TgC_month_' + model)

    if  var=='ra' or var=='rh'or var == 'ra+rh' or var=='ra+rh-gpp' or var=='nbp-(ra+rh)+gpp':
        for model in models_bad_low_amplitude_new_for_ra_rh_mean:
            column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
    elif var=='fFire' or var=='ra+rh-gpp+fFire' or var=='nbp-(ra+rh-gpp+fFire)':
        for model in models_bad_low_amplitude_new_for_ra_rh_fFire:
            column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
    else:
        for model in models_bad_low_amplitude_new:
            column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
    
    if var=='npp':
        for model in models_bad_high_amplitude_new_for_npp_mean:
            column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
    elif var=='fFire' or var=='ra+rh-gpp+fFire' or var=='nbp-(ra+rh-gpp+fFire)':
        for model in models_bad_high_amplitude_new_for_fFire:
            column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
    else:
        for model in models_bad_high_amplitude_new:
            column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
    
    df_TRENDY = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var, unit='mean_of_variable', start_year=2009, end_year=2019)
    #print(df_TRENDY.head())
    print(var)
    df_TRENDY_modified_very_good = df_TRENDY[['MonthDate', 'Month']]
    df_TRENDY_modified_very_good['mean'] = df_TRENDY[column_name_list_very_good].mean(axis=1)
    df_TRENDY_modified_very_good['std'] = df_TRENDY[column_name_list_very_good].std(ddof=0, axis=1)
    df_TRENDY_modified_very_good.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_very_good_models/Mean_very_good_models_for_'+var+'_SAT_721.pkl')
    del df_TRENDY_modified_very_good
    del column_name_list_very_good
    df_TRENDY_modified_ok = df_TRENDY[['MonthDate', 'Month']]
    df_TRENDY_modified_ok['mean'] = df_TRENDY[column_name_list_ok].mean(axis=1)
    df_TRENDY_modified_ok['std'] = df_TRENDY[column_name_list_ok].std(ddof=0, axis=1)
    df_TRENDY_modified_ok.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_ok_models/Mean_ok_models_for_'+var+'_SAT_721.pkl')
    del df_TRENDY_modified_ok
    del column_name_list_ok
    df_TRENDY_modified_bad_low = df_TRENDY[['MonthDate', 'Month']]
    df_TRENDY_modified_bad_low['mean'] = df_TRENDY[column_name_list_bad_low].mean(axis=1)
    df_TRENDY_modified_bad_low['std'] = df_TRENDY[column_name_list_bad_low].std(ddof=0, axis=1)
    df_TRENDY_modified_bad_low.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_bad_low_ampl_models/Mean_bad_low_amplitude_models_for_'+var+'_SAT_721.pkl')
    del df_TRENDY_modified_bad_low
    del column_name_list_bad_low
    df_TRENDY_modified_bad_high = df_TRENDY[['MonthDate', 'Month']]
    df_TRENDY_modified_bad_high['mean'] = df_TRENDY[column_name_list_bad_high].mean(axis=1)
    df_TRENDY_modified_bad_high['std'] = df_TRENDY[column_name_list_bad_high].std(ddof=0, axis=1)
    df_TRENDY_modified_bad_high.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_bad_high_ampl_models/Mean_bad_high_amplitude_models_for_'+var+'_SAT_721.pkl')
    del df_TRENDY_modified_bad_high
    del column_name_list_bad_high
    


# For TRENDY arid/humid region

## Calculate the mean of a variable including only the very_good & ok model category

In [ ]:
import numpy as np
import pandas as pd
import geopandas
import datetime
import warnings
warnings.filterwarnings('ignore')
import functions_to_load_datasets as load_datasets

# NEW model categories
models_very_good_list = ['CLASSIC', 'OCN']
models_very_good_list_for_fFire = ['CLASSIC']
models_ok_list = ['ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE']
models_ok_list_for_fFire = ['ISBA-CTRIP']

Variables = ['nbp', 'gpp', 'npp', 'ra', 'rh', 'fFire']
Variables_new = ['nbp', 'gpp', 'npp', 'ra', 'rh', 'ra+rh', 'ra+rh-gpp', 'nbp-(ra+rh)+gpp']
#Variable = ['fFire', 'ra+rh-gpp+fFire']
#Variable = ['nbp-(ra+rh-gpp+fFire)'] # works only for vg & ok models

arid_type_list = ['looselimit_arid', 'looselimit_humid', 'strictlimit_arid', 'strictlimit_humid']
#arid_type_list = ['looselimit_arid']
for aridity in arid_type_list:
    print(aridity)
    # generate files for ensemble mean:
    for var in Variables_new:
        print(var)
        
        column_name_list_very_good = []
        column_name_list_ok = []
        
        if var == 'ra+rh':
            variable = 'ratot+rh'
        elif var=='ra+rh-gpp':
            variable = 'ratot+rhtot-gpp'
        elif var=='nbp-(ra+rh)+gpp':
            variable = 'nbptot-(ratot+rhtot)+gpp'
        elif var=='ra+rh-gpp+fFire':
            variable = 'ratot+rhtot-gpptot+fFire'
        elif var=='nbp-(ra+rh-gpp+fFire)':
            variable = 'nbptot-(ratot+rhtot-gpptot+fFiretot)'
        else:
            variable = var
        
        if var=='fFire' or var=='ra+rh-gpp+fFire':
            for model in models_very_good_list:
                column_name_list_very_good.append(variable + 'tot_TgC_month_' + model)
        elif var=='nbp-(ra+rh-gpp+fFire)':
            for model in models_very_good_list:
                column_name_list_very_good.append(variable + '_TgC_month_' + model)
        else:
            for model in models_very_good_list:
                column_name_list_very_good.append(variable + 'tot_TgC_month_' + model)
        
        if var=='fFire' or var=='ra+rh-gpp+fFire':
            for model in models_ok_list:
                column_name_list_ok.append(variable + 'tot_TgC_month_' + model)
        elif var=='nbp-(ra+rh-gpp+fFire)':
            for model in models_ok_list:
                column_name_list_ok.append(variable + '_TgC_month_' + model)
        else:
            for model in models_ok_list:
                column_name_list_ok.append(variable + 'tot_TgC_month_' + model)

        
        df_TRENDY = load_datasets.load_TRENDY_model_gdf(variable=var, unit='mean_of_variable', arid_type=aridity, start_year=2009)
        
        df_TRENDY_modified_very_good = df_TRENDY[['MonthDate', 'Month']]
        df_TRENDY_modified_very_good['mean'] = df_TRENDY[column_name_list_very_good].mean(axis=1)
        df_TRENDY_modified_very_good['std'] = df_TRENDY[column_name_list_very_good].std(ddof=0, axis=1)
        df_TRENDY_modified_very_good.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_very_good_models/Mean_very_good_models_for_'+var+'_SAT_'+aridity+'.pkl')
        del df_TRENDY_modified_very_good
        del column_name_list_very_good
        df_TRENDY_modified_ok = df_TRENDY[['MonthDate', 'Month']]
        df_TRENDY_modified_ok['mean'] = df_TRENDY[column_name_list_ok].mean(axis=1)
        df_TRENDY_modified_ok['std'] = df_TRENDY[column_name_list_ok].std(ddof=0, axis=1)
        df_TRENDY_modified_ok.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_ok_models/Mean_ok_models_for_'+var+'_SAT_'+aridity+'.pkl')
        del df_TRENDY_modified_ok
        del column_name_list_ok
        
        

### for bad high ampl models and bad low ampl models

In [ ]:
import numpy as np
import pandas as pd
import geopandas
import datetime
import warnings
warnings.filterwarnings('ignore')
import functions_to_load_datasets as load_datasets

#new model categories
models_bad_low_amplitude_new = ['ORCHIDEE-CNP', 'LPJ', 'JULES-ES-1p0', 'JSBACH', 'IBIS']
models_bad_low_amplitude_new_for_ra_rh_mean = ['LPJ', 'JSBACH', 'IBIS'] # all models containing respiration & without ORCHIDEE-CNP because it is very bad for respiration
models_bad_low_amplitude_new_for_ra_rh_fFire = ['LPJ', 'JSBACH']
models_bad_high_amplitude_new = ['SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
models_bad_high_amplitude_new_for_npp_mean = ['SDGVM', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3'] # all models except VISIT because this models does not have npp
models_bad_high_amplitude_new_for_fFire = ['SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0']

Variables = ['gpp', 'nbp', 'npp', 'ra', 'rh', 'fFire']
Variables_new = ['gpp', 'nbp', 'npp', 'ra', 'rh', 'ra+rh', 'ra+rh-gpp', 'nbp-(ra+rh)+gpp']

arid_type_list = ['looselimit_arid', 'looselimit_humid', 'strictlimit_arid', 'strictlimit_humid']
#arid_type_list = ['looselimit_arid']
for aridity in arid_type_list:
    print(aridity)
    # generate files for ensemble mean:
    for var in Variables_new:
        column_name_list_bad_low = []
        column_name_list_bad_high = []
        
        if var == 'ra+rh':
            variable = 'ratot+rh'
        elif var=='ra+rh-gpp':
            variable = 'ratot+rhtot-gpp'
        elif var=='nbp-(ra+rh)+gpp':
            variable = 'nbptot-(ratot+rhtot)+gpp'
        elif var=='ra+rh-gpp+fFire':
            variable = 'ratot+rhtot-gpptot+fFire'
        elif var=='nbp-(ra+rh-gpp+fFire)':
            variable = 'nbptot-(ratot+rhtot-gpptot+fFiretot)'
        else:
            variable = var
        
        
        if  var=='ra' or var=='rh'or var == 'ra+rh' or var=='ra+rh-gpp' or var=='nbp-(ra+rh)+gpp':
            for model in models_bad_low_amplitude_new_for_ra_rh_mean:
                column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
        elif var=='fFire' or var=='ra+rh-gpp+fFire' or var=='nbp-(ra+rh-gpp+fFire)':
            for model in models_bad_low_amplitude_new_for_ra_rh_fFire:
                column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
        else:
            for model in models_bad_low_amplitude_new:
                column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
        
        if var=='npp':
            for model in models_bad_high_amplitude_new_for_npp_mean:
                column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
        elif var=='fFire' or var=='ra+rh-gpp+fFire' or var=='nbp-(ra+rh-gpp+fFire)':
            for model in models_bad_high_amplitude_new_for_fFire:
                column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
        else:
            for model in models_bad_high_amplitude_new:
                column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
        
        df_TRENDY = load_datasets.load_TRENDY_model_gdf(variable=var, unit='mean_of_variable', arid_type=aridity, start_year=2009)
        print(var)
        
        df_TRENDY_modified_bad_low = df_TRENDY[['MonthDate', 'Month']]
        df_TRENDY_modified_bad_low['mean'] = df_TRENDY[column_name_list_bad_low].mean(axis=1)
        df_TRENDY_modified_bad_low['std'] = df_TRENDY[column_name_list_bad_low].std(ddof=0, axis=1)
        df_TRENDY_modified_bad_low.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_bad_low_ampl_models/Mean_bad_low_amplitude_models_for_'+var+'_SAT_'+aridity+'.pkl')
        del df_TRENDY_modified_bad_low
        del column_name_list_bad_low
        df_TRENDY_modified_bad_high = df_TRENDY[['MonthDate', 'Month']]
        df_TRENDY_modified_bad_high['mean'] = df_TRENDY[column_name_list_bad_high].mean(axis=1)
        df_TRENDY_modified_bad_high['std'] = df_TRENDY[column_name_list_bad_high].std(ddof=0, axis=1)
        df_TRENDY_modified_bad_high.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_bad_high_ampl_models/Mean_bad_high_amplitude_models_for_'+var+'_SAT_'+aridity+'.pkl')
        del df_TRENDY_modified_bad_high
        del column_name_list_bad_high

# For TRENDY model arid subregions

## Calculate the mean of a variable including only the very_good & ok model category

In [ ]:
import numpy as np
import pandas as pd
import geopandas
import datetime
import warnings
warnings.filterwarnings('ignore')
import functions_to_load_datasets as load_datasets

# NEW model categories
models_very_good_list = ['CLASSIC', 'OCN']
models_very_good_list_for_fFire = ['CLASSIC']
models_ok_list = ['ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE']
models_ok_list_for_fFire = ['ISBA-CTRIP']

Variables = ['nbp', 'gpp', 'npp', 'ra', 'rh', 'fFire']
Variables_new = ['nbp', 'gpp', 'npp', 'ra', 'rh', 'ra+rh', 'ra+rh-gpp', 'nbp-(ra+rh)+gpp']
#Variable = ['fFire', 'ra+rh-gpp+fFire']
#Variable = ['nbp-(ra+rh-gpp+fFire)'] # works only for vg & ok models

arid_type_list = ['looselimit_arid_west', 'looselimit_arid_east', 'strictlimit_arid_west', 'strictlimit_arid_east']
#arid_type_list = ['looselimit_arid_west']
for aridity in arid_type_list:
    print(aridity)
    # generate files for ensemble mean:
    for var in Variables_new:
        print(var)
        
        column_name_list_very_good = []
        column_name_list_ok = []
        
        if var == 'ra+rh':
            variable = 'ratot+rh'
        elif var=='ra+rh-gpp':
            variable = 'ratot+rhtot-gpp'
        elif var=='nbp-(ra+rh)+gpp':
            variable = 'nbptot-(ratot+rhtot)+gpp'
        elif var=='ra+rh-gpp+fFire':
            variable = 'ratot+rhtot-gpptot+fFire'
        elif var=='nbp-(ra+rh-gpp+fFire)':
            variable = 'nbptot-(ratot+rhtot-gpptot+fFiretot)'
        else:
            variable = var
        
        if var=='fFire' or var=='ra+rh-gpp+fFire':
            for model in models_very_good_list:
                column_name_list_very_good.append(variable + 'tot_TgC_month_' + model)
        elif var=='nbp-(ra+rh-gpp+fFire)':
            for model in models_very_good_list:
                column_name_list_very_good.append(variable + '_TgC_month_' + model)
        else:
            for model in models_very_good_list:
                column_name_list_very_good.append(variable + 'tot_TgC_month_' + model)
        
        if var=='fFire' or var=='ra+rh-gpp+fFire':
            for model in models_ok_list:
                column_name_list_ok.append(variable + 'tot_TgC_month_' + model)
        elif var=='nbp-(ra+rh-gpp+fFire)':
            for model in models_ok_list:
                column_name_list_ok.append(variable + '_TgC_month_' + model)
        else:
            for model in models_ok_list:
                column_name_list_ok.append(variable + 'tot_TgC_month_' + model)

        
        df_TRENDY = load_datasets.load_TRENDY_model_gdf(variable=var, unit='mean_of_variable', arid_type=aridity, start_year=2009)
        
        df_TRENDY_modified_very_good = df_TRENDY[['MonthDate', 'Month']]
        df_TRENDY_modified_very_good['mean'] = df_TRENDY[column_name_list_very_good].mean(axis=1)
        df_TRENDY_modified_very_good['std'] = df_TRENDY[column_name_list_very_good].std(ddof=0, axis=1)
        df_TRENDY_modified_very_good.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_very_good_models/Mean_very_good_models_for_'+var+'_SAT_'+aridity+'.pkl')
        del df_TRENDY_modified_very_good
        del column_name_list_very_good
        df_TRENDY_modified_ok = df_TRENDY[['MonthDate', 'Month']]
        df_TRENDY_modified_ok['mean'] = df_TRENDY[column_name_list_ok].mean(axis=1)
        df_TRENDY_modified_ok['std'] = df_TRENDY[column_name_list_ok].std(ddof=0, axis=1)
        df_TRENDY_modified_ok.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_ok_models/Mean_ok_models_for_'+var+'_SAT_'+aridity+'.pkl')
        del df_TRENDY_modified_ok
        del column_name_list_ok

## for bad model categories

In [1]:
import numpy as np
import pandas as pd
import geopandas
import datetime
import warnings
warnings.filterwarnings('ignore')
import functions_to_load_datasets as load_datasets

#new model categories
models_bad_low_amplitude_new = ['ORCHIDEE-CNP', 'LPJ', 'JULES-ES-1p0', 'JSBACH', 'IBIS']
models_bad_low_amplitude_new_for_ra_rh_mean = ['LPJ', 'JSBACH', 'IBIS'] # all models containing respiration & without ORCHIDEE-CNP because it is very bad for respiration
models_bad_low_amplitude_new_for_ra_rh_fFire = ['LPJ', 'JSBACH']
models_bad_high_amplitude_new = ['SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
models_bad_high_amplitude_new_for_npp_mean = ['SDGVM', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3'] # all models except VISIT because this models does not have npp
models_bad_high_amplitude_new_for_fFire = ['SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0']

Variables = ['gpp', 'nbp', 'npp', 'ra', 'rh', 'fFire']
Variables_new = ['gpp', 'nbp', 'npp', 'ra', 'rh', 'ra+rh', 'ra+rh-gpp', 'nbp-(ra+rh)+gpp']

arid_type_list = ['looselimit_arid_west', 'looselimit_arid_east', 'strictlimit_arid_west', 'strictlimit_arid_east']
#arid_type_list = ['looselimit_arid_west']
for aridity in arid_type_list:
    print(aridity)
    # generate files for ensemble mean:
    for var in Variables_new:
        column_name_list_bad_low = []
        column_name_list_bad_high = []
        
        if var == 'ra+rh':
            variable = 'ratot+rh'
        elif var=='ra+rh-gpp':
            variable = 'ratot+rhtot-gpp'
        elif var=='nbp-(ra+rh)+gpp':
            variable = 'nbptot-(ratot+rhtot)+gpp'
        elif var=='ra+rh-gpp+fFire':
            variable = 'ratot+rhtot-gpptot+fFire'
        elif var=='nbp-(ra+rh-gpp+fFire)':
            variable = 'nbptot-(ratot+rhtot-gpptot+fFiretot)'
        else:
            variable = var
        
        
        if  var=='ra' or var=='rh'or var == 'ra+rh' or var=='ra+rh-gpp' or var=='nbp-(ra+rh)+gpp':
            for model in models_bad_low_amplitude_new_for_ra_rh_mean:
                column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
        elif var=='fFire' or var=='ra+rh-gpp+fFire' or var=='nbp-(ra+rh-gpp+fFire)':
            for model in models_bad_low_amplitude_new_for_ra_rh_fFire:
                column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
        else:
            for model in models_bad_low_amplitude_new:
                column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
        
        if var=='npp':
            for model in models_bad_high_amplitude_new_for_npp_mean:
                column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
        elif var=='fFire' or var=='ra+rh-gpp+fFire' or var=='nbp-(ra+rh-gpp+fFire)':
            for model in models_bad_high_amplitude_new_for_fFire:
                column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
        else:
            for model in models_bad_high_amplitude_new:
                column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
        
        df_TRENDY = load_datasets.load_TRENDY_model_gdf(variable=var, unit='mean_of_variable', arid_type=aridity, start_year=2009)
        print(var)
        
        df_TRENDY_modified_bad_low = df_TRENDY[['MonthDate', 'Month']]
        df_TRENDY_modified_bad_low['mean'] = df_TRENDY[column_name_list_bad_low].mean(axis=1)
        df_TRENDY_modified_bad_low['std'] = df_TRENDY[column_name_list_bad_low].std(ddof=0, axis=1)
        df_TRENDY_modified_bad_low.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_bad_low_amplitude_models/Mean_bad_low_amplitude_models_for_'+var+'_SAT_'+aridity+'.pkl')
        del df_TRENDY_modified_bad_low
        del column_name_list_bad_low
        df_TRENDY_modified_bad_high = df_TRENDY[['MonthDate', 'Month']]
        df_TRENDY_modified_bad_high['mean'] = df_TRENDY[column_name_list_bad_high].mean(axis=1)
        df_TRENDY_modified_bad_high['std'] = df_TRENDY[column_name_list_bad_high].std(ddof=0, axis=1)
        df_TRENDY_modified_bad_high.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_bad_high_amplitude_models/Mean_bad_high_amplitude_models_for_'+var+'_SAT_'+aridity+'.pkl')
        del df_TRENDY_modified_bad_high
        del column_name_list_bad_high

ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


looselimit_arid_west
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
gpp
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
nbp
Done loading data for gdf1x1_CABLE-POP_npp_SAT_721.pkl
npp
Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
ra
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
rh
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
ra+rh
Done loading data for gdf1x1_CABLE-POP_ra+rh-gpp_SAT_721.pkl
ra+rh-gpp
Done loading data for gdf1x1_CABLE-POP_nbp-(ra+rh)+gpp_SAT_721.pkl
nbp-(ra+rh)+gpp
looselimit_arid_east
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
gpp
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
nbp
Done loading data for gdf1x1_CABLE-POP_npp_SAT_721.pkl
npp
Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
ra
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
rh
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
ra+rh
Done loading data for gdf1x1_CABLE-POP_ra+rh-gpp_SAT_721.pkl
ra+rh-gpp
Done loading data for gdf

### old model category version

In [2]:
models_good_list = ['ORCHIDEE', 'OCN', 'ISBA-CTRIP', 'ISAM', 'CLASSIC']
models_bad_high_amplitude = ['SDGVM', 'YIBs', 'VISIT', 'ORCHIDEEv3', 'LPX-Bern', 'CLM5.0']
models_bad_high_amplitude_for_mean_npp = ['SDGVM', 'YIBs', 'ORCHIDEEv3', 'LPX-Bern', 'CLM5.0'] # use for mean of npp;  all models except VISIT because this models does not have npp
models_bad_low_amplitude = ['ORCHIDEE-CNP', 'LPJ', 'JULES-ES-1p0', 'JSBACH', 'IBIS']
models_bad_low_amplitude_ra_rh = ['ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS'] # all models containing respiration
models_bad_low_amplitude_for_mean = ['LPJ', 'JSBACH', 'IBIS'] # use for mean of respireation; all models containing GOOD quality respiration

Variables = ['gpp', 'nbp', 'npp', 'ra', 'rh', 'fFire']

column_name_list_good = []
column_name_list_bad_low = []
column_name_list_bad_high = []
    
# generate files for ensemble mean:
for var in ['gpp']:
    if var == 'ra+rh':
        variable = 'ratot+rh'
    else:
        variable = var
    for model in models_good_list:
        column_name_list_good.append(variable + 'tot_TgC_month_' + model)
    for model in models_bad_low_amplitude_for_mean:
        column_name_list_bad_low.append(variable + 'tot_TgC_month_' + model)
    for model in models_bad_high_amplitude:
        column_name_list_bad_high.append(variable + 'tot_TgC_month_' + model)
    
    df_TRENDY = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var, unit='mean_of_variable', start_year=2009, end_year=2019)
    #print(df_TRENDY.head())
    print(var)
    df_TRENDY_modified_good = df_TRENDY[['MonthDate', 'Month']]
    df_TRENDY_modified_good['mean'] = df_TRENDY[column_name_list_good].mean(axis=1)
    df_TRENDY_modified_good['std'] = df_TRENDY[column_name_list_good].std(ddof=0, axis=1)
    df_TRENDY_modified_good.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_good_models/Mean_good_models_for_'+var+'_SAT_721.pkl')
    del df_TRENDY_modified_good
    #del column_name_list_good
    df_TRENDY_modified_bad_low = df_TRENDY[['MonthDate', 'Month']]
    df_TRENDY_modified_bad_low['mean'] = df_TRENDY[column_name_list_bad_low].mean(axis=1)
    df_TRENDY_modified_bad_low['std'] = df_TRENDY[column_name_list_bad_low].std(ddof=0, axis=1)
    df_TRENDY_modified_bad_low.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_bad_low_ampl_models/Mean_bad_low_amplitude_models_for_'+var+'_SAT_721.pkl')
    del df_TRENDY_modified_bad_low
    #del column_name_list_bad_low
    df_TRENDY_modified_bad_high = df_TRENDY[['MonthDate', 'Month']]
    df_TRENDY_modified_bad_high['mean'] = df_TRENDY[column_name_list_bad_high].mean(axis=1)
    df_TRENDY_modified_bad_high['std'] = df_TRENDY[column_name_list_bad_high].std(ddof=0, axis=1)
    df_TRENDY_modified_bad_high.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/mean_of_variable_bad_high_ampl_models/Mean_bad_high_amplitude_models_for_'+var+'_SAT_721.pkl')
    del df_TRENDY_modified_bad_high
    #del column_name_list_bad_high


Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
ra+rh


## Other

In [9]:
print(datetime.datetime(2008,1,1,0,0,0))
print(datetime.date(2008,1,1))
print(datetime.date(2008,1,1)==datetime.datetime(2008,1,1,0,0,0))

2008-01-01 00:00:00
2008-01-01
False


In [3]:
df = pd.read_pickle('/mnt/data/users/lartelt/MA/TRENDY/CABLE-POP/gdf1x1__npp_SAT_721.pkl')
print(min(df.npp.values))
print(df)

nan
        level_0  npptot   MonthDate        Date   Day  Month    Year  index  \
0         16226     NaN  2008-01-15  2008-01-12  12.0    1.0  2008.0  -59.5   
1         16227     NaN  2008-01-15  2008-01-12  12.0    1.0  2008.0  -58.5   
2         16228     NaN  2008-01-15  2008-01-12  12.0    1.0  2008.0  -57.5   
3         16229     NaN  2008-01-15  2008-01-12  12.0    1.0  2008.0  -56.5   
4         16230     NaN  2008-01-15  2008-01-12  12.0    1.0  2008.0  -55.5   
...         ...     ...         ...         ...   ...    ...     ...    ...   
561595    27801     NaN  2019-12-15  2019-12-09   9.0   12.0  2019.0   -4.5   
561596    27802     NaN  2019-12-15  2019-12-09   9.0   12.0  2019.0   -3.5   
561597    27803     NaN  2019-12-15  2019-12-09   9.0   12.0  2019.0   -2.5   
561598    27804     NaN  2019-12-15  2019-12-09   9.0   12.0  2019.0   -1.5   
561599    27805     NaN  2019-12-15  2019-12-09   9.0   12.0  2019.0   -0.5   

        Long  npp   Lat                     geo

In [16]:
#python program to check if a directory exists
TrendyModels_all = ['CABLE-POP', 'CLASSIC', 'DLEM', 'IBIS', 'ISAM', 'JSBACH', 
                'LPX-Bern', 'OCN', 'ORCHIDEE', 'ORCHIDEE-CNP', 'ORCHIDEEv3', 
                'LPJ-GUESS','CLM5.0','ISBA-CTRIP','JULES-ES-1p0','LPJ','SDGVM',
                'VISIT','YIBs']

TrendyModels_working = ['CABLE-POP', 'CLASSIC', 'DLEM', 'IBIS', 'JSBACH', 
                        'LPX-Bern', 'OCN', 'ORCHIDEE', 'ORCHIDEE-CNP', 'ORCHIDEEv3', 
                        'LPJ-GUESS','ISBA-CTRIP','LPJ','SDGVM','VISIT', 'YIBs']

TrendyModels_working_and_noleapcalendar = ['DLEM', 'LPX-Bern', 'ORCHIDEE', 'ORCHIDEE-CNP']
'''
for model in TrendyModels_working:
    path = "/mnt/data/users/lartelt/MA/TRENDY/"+model+"/"
    print(model)
    # Check whether the specified path exists or not
    df = pd.read_pickle(path+'gdf1x1__gpp_SAT_721.pkl')
    # check if value is nan
    if np.isnan(max(df.gpp.values)) and np.isnan(min(df.gpp.values)):
        print(model, ' npp values are NaN')
'''
for model in TrendyModels_working:
    path = "/mnt/data/users/lartelt/MA/TRENDY/"+model+"/"
    df = pd.read_pickle(path+'gdf1x1__gpp_SAT_721.pkl')
    print(model)
    print(min(df.Date))
    

CABLE-POP
2008-01-12
CLASSIC
2008-01-31
DLEM
2008-01-15
IBIS
2008-01-15
JSBACH
2008-01-16
LPX-Bern
2008-02-01
OCN
2008-01-31
ORCHIDEE
2008-01-16
ORCHIDEE-CNP
2008-01-16
ORCHIDEEv3
2008-01-16
LPJ-GUESS
2008-01-16
ISBA-CTRIP
2008-01-15
LPJ
2008-02-01
SDGVM
2008-01-16
VISIT
2008-01-15
YIBs
2008-01-15


In [2]:
df = pd.read_pickle('/mnt/data/users/lartelt/MA/TRENDY/CABLE-POP/gdf1x1__gpp_SAT_721.pkl')
df
#plot_func.plotGDFspatially_simple_world(df, savepath_w_savename='/mnt/data/users/lartelt/MA/TRENDY/CLM5.0/map_CLM_gpp_SAT_721.png')

,level_0,gpptot,MonthDate,Date,Day,Month,Year,index,Long,gpp,Lat,geometry,Area
0,18614,787.793053,2008-01-15,2008-01-12,12.0,1.0,2008.0,-11.5,-76.5,6.527823e-08,-11.5,POINT (-76.50000 -11.50000),1.206824e+10
1,18791,0.000000,2008-01-15,2008-01-12,12.0,1.0,2008.0,-14.5,-75.5,0.000000e+00,-14.5,POINT (-75.50000 -14.50000),1.192686e+10
2,18792,132.042195,2008-01-15,2008-01-12,12.0,1.0,2008.0,-13.5,-75.5,1.102413e-08,-13.5,POINT (-75.50000 -13.50000),1.197756e+10
3,18793,505.675714,2008-01-15,2008-01-12,12.0,1.0,2008.0,-12.5,-75.5,4.205312e-08,-12.5,POINT (-75.50000 -12.50000),1.202469e+10
4,18794,1027.376760,2008-01-15,2008-01-12,12.0,1.0,2008.0,-11.5,-75.5,8.513065e-08,-11.5,POINT (-75.50000 -11.50000),1.206824e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120667,25820,68.449250,2019-12-15,2019-12-09,9.0,12.0,2019.0,-5.5,-36.5,5.585980e-09,-5.5,POINT (-36.50000 -5.50000),1.225376e+10
120668,25997,970.962952,2019-12-15,2019-12-09,9.0,12.0,2019.0,-8.5,-35.5,7.973563e-08,-8.5,POINT (-35.50000 -8.50000),1.217728e+10
120669,25998,1065.314265,2019-12-15,2019-12-09,9.0,12.0,2019.0,-7.5,-35.5,8.727506e-08,-7.5,POINT (-35.50000 -7.50000),1.220640e+10
120670,25999,753.406742,2019-12-15,2019-12-09,9.0,12.0,2019.0,-6.5,-35.5,6.159362e-08,-6.5,POINT (-35.50000 -6.50000),1.223189e+10


## Calculate ra+rh from TRENDY ra + TRENDY rh

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import glob
import os
import datetime
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

var1='ra'
var2='rh'
df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var1, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
#print(df_TRENDY_var1.head())
df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var2, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
#print(df_TRENDY_var2.head())
#df_TRENDY = pd.merge(df_TRENDY_var1, df_TRENDY_var2, on=['MonthDate', 'Month', 'Year'], how='outer')
df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

model_list = []
for column in columns_var1:
    col = 'rh'+column[2:]
    if col in columns_var2:
        model_list.append(column[16:])

for model in model_list:
    df_TRENDY_sum[var1+'tot+'+var2+'tot_TgC_month_'+model] = df_TRENDY_var1[var1+'tot_TgC_month_'+model] + df_TRENDY_var2[var2+'tot_TgC_month_'+model]

df_TRENDY_sum_old = df_TRENDY_sum.copy()
df_TRENDY_sum['mean'] = df_TRENDY_sum.mean(axis=1)
df_TRENDY_sum['std'] = df_TRENDY_sum_old.std(axis=1)
df_TRENDY_sum['count'] = df_TRENDY_sum_old.count(axis=1)
#print(df_TRENDY_sum.head())
#save df_TRENDY_sum to pickle
df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh_SAT_721.pkl')


## Calculate ra+rh-gpp+fFire

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import glob
import os
import datetime
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

var1='ra+rh-gpp'
var1_long = 'ratot+rhtot-gpptot'
var2='fFire'
df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var1, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var1.head())
df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var2, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var2.head())
#df_TRENDY = pd.merge(df_TRENDY_var1, df_TRENDY_var2, on=['MonthDate', 'Month', 'Year'], how='outer')
df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

model_list = []
for column in columns_var1:
    col = 'fFiretot'+column[len(var1_long):]
    print(col)
    if col in columns_var2:
        model_list.append(column[len(var1_long)+11:])
print(model_list)

for model in model_list:
    df_TRENDY_sum[var1_long+'+'+var2+'tot_TgC_month_'+model] = df_TRENDY_var1[var1_long+'_TgC_month_'+model] + df_TRENDY_var2[var2+'tot_TgC_month_'+model]
#print(df_TRENDY_sum)
df_TRENDY_sum_old = df_TRENDY_sum.copy()
df_TRENDY_sum['mean'] = df_TRENDY_sum.mean(axis=1)
df_TRENDY_sum['std'] = df_TRENDY_sum_old.std(axis=1)
df_TRENDY_sum['count'] = df_TRENDY_sum_old.count(axis=1)
print(df_TRENDY_sum.head())
#save df_TRENDY_sum to pickle
df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh-gpp+fFire_SAT_721.pkl')


ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


Done loading data for gdf1x1_CABLE-POP_ra+rh-gpp_SAT_721.pkl
    MonthDate  Month  Year  ratot+rhtot-gpptot_TgC_month_CABLE-POP  \
0  2008-01-15      1  2008                               10.794904   
1  2008-02-15      2  2008                               13.556841   
2  2008-03-15      3  2008                                4.967674   
3  2008-04-15      4  2008                              -69.607646   
4  2008-05-15      5  2008                             -149.008510   

   ratot+rhtot-gpptot_TgC_month_CLASSIC  ratot+rhtot-gpptot_TgC_month_DLEM  \
0                            -41.154891                        -560.100223   
1                           -128.051296                        -626.054699   
2                           -197.729914                        -581.359404   
3                           -194.655370                        -406.694386   
4                           -152.837684                         -79.959368   

   ratot+rhtot-gpptot_TgC_month_IBIS  ratot+rhtot

## Calculate nbp - (ra+rh-gpp+fFire)

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import glob
import os
import datetime
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

var1='ra+rh-gpp+fFire'
var1_long = 'ratot+rhtot-gpptot+fFiretot'
var2='nbp'
df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var1, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var1.head())
df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var2, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var2.head())
#df_TRENDY = pd.merge(df_TRENDY_var1, df_TRENDY_var2, on=['MonthDate', 'Month', 'Year'], how='outer')
df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

model_list = []
for column in columns_var1:
    col = 'nbptot'+column[len(var1_long):]
    print(col)
    if col in columns_var2:
        model_list.append(column[len(var1)+23:])
print(model_list)

for model in model_list:
    df_TRENDY_sum[var2+'tot'+'-('+var1_long+')_TgC_month_'+model] = df_TRENDY_var2[var2+'tot_TgC_month_'+model] - df_TRENDY_var1[var1_long+'_TgC_month_'+model]
#print(df_TRENDY_sum)
df_TRENDY_sum_old = df_TRENDY_sum.copy()
df_TRENDY_sum['mean'] = df_TRENDY_sum.mean(axis=1)
df_TRENDY_sum['std'] = df_TRENDY_sum_old.std(axis=1)
df_TRENDY_sum['count'] = df_TRENDY_sum_old.count(axis=1)
print(df_TRENDY_sum.head())
#save df_TRENDY_sum to pickle
df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_nbp-(ra+rh-gpp+fFire)_SAT_721.pkl')


ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


Done loading data for gdf1x1_CABLE-POP_ra+rh-gpp+fFire_SAT_721.pkl
    MonthDate  Month  Year  ratot+rhtot-gpptot+fFiretot_TgC_month_CLASSIC  \
0  2008-01-15      1  2008                                     -23.903336   
1  2008-02-15      2  2008                                    -118.711112   
2  2008-03-15      3  2008                                    -191.201703   
3  2008-04-15      4  2008                                    -187.245828   
4  2008-05-15      5  2008                                    -147.160837   

   ratot+rhtot-gpptot+fFiretot_TgC_month_ISAM  \
0                                 -178.473153   
1                                 -105.450365   
2                                  -45.335060   
3                                    9.144359   
4                                   21.871161   

   ratot+rhtot-gpptot+fFiretot_TgC_month_JSBACH  \
0                                     68.961615   
1                                     28.745596   
2                     

## insert column with zeros for missing models

In [2]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import glob
import os
import datetime
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

start_year = None
end_year   = None
var='fFire'
TRENDY_model_category = 'very_good_models'
df_TRENDY = pd.read_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_fFire_SAT_721_NOT_included_zeros_for_missing_vg_and_ok_models.pkl')

df_TRENDY

,MonthDate,fFiretot_TgC_month_CLASSIC,fFiretot_TgC_month_JSBACH,fFiretot_TgC_month_LPX-Bern,fFiretot_TgC_month_ORCHIDEE,fFiretot_TgC_month_ORCHIDEE-CNP,fFiretot_TgC_month_CLM5.0,fFiretot_TgC_month_ISBA-CTRIP,fFiretot_TgC_month_LPJ,fFiretot_TgC_month_SDGVM,fFiretot_TgC_month_VISIT,mean,std,count,Month
0,2008-01-15,17.251555,14.278623,20.144920,0.0,0.0,25.036818,20.253562,25.344799,37.862121,18.005945,17.817834,10.815373,10,1
1,2008-02-15,9.340184,21.739282,20.864381,0.0,0.0,21.552878,13.239303,26.249970,35.419403,16.449389,16.485479,10.636873,10,2
2,2008-03-15,6.528211,32.439310,20.144920,0.0,0.0,13.916302,9.857798,25.344799,37.862121,21.342671,16.743613,12.326827,10,3
3,2008-04-15,7.409542,31.586002,20.144920,0.0,0.0,7.807634,14.991987,25.344799,36.640762,22.742262,16.666791,12.117793,10,4
4,2008-05-15,5.676847,49.469675,20.144920,0.0,0.0,6.314067,19.867079,25.344799,37.862121,25.727183,19.040669,15.595440,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2019-08-15,72.452662,40.737665,20.719732,0.0,0.0,30.532673,100.084512,22.562840,33.529802,24.158951,34.477884,29.354976,10,8
140,2019-09-15,88.235119,50.136619,20.719733,0.0,0.0,63.973931,126.734333,22.562840,32.448196,25.190528,43.000130,38.195677,10,9
141,2019-10-15,47.411970,31.741872,20.719732,0.0,0.0,100.163997,66.004480,22.562840,33.529802,25.064699,34.719939,28.776715,10,10
142,2019-11-15,18.577060,21.131917,20.719733,0.0,0.0,74.819297,23.489458,22.562840,32.448196,22.152184,23.590068,19.642880,10,11


In [3]:
models_very_good_list = ['CLASSIC', 'OCN']
models_very_good_list_for_fFire = ['CLASSIC'] 
models_ok_list = ['ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE']
models_ok_list_for_fFire = ['ISBA-CTRIP', 'ORCHIDEE']


df_TRENDY['fFiretot_TgC_month_OCN'] = np.zeros(len(df_TRENDY.Month.values))
df_TRENDY['fFiretot_TgC_month_YIBs'] = np.zeros(len(df_TRENDY.Month.values))
df_TRENDY['fFiretot_TgC_month_ISAM'] = np.zeros(len(df_TRENDY.Month.values))
df_TRENDY.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_fFire_SAT_721.pkl')

In [4]:
df = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable='fFire', unit='mean_of_variable', start_year=start_year, end_year=end_year)
df

Done loading data for gdf1x1_CABLE-POP_fFire_SAT_721.pkl


,MonthDate,fFiretot_TgC_month_CLASSIC,fFiretot_TgC_month_JSBACH,fFiretot_TgC_month_LPX-Bern,fFiretot_TgC_month_ORCHIDEE,fFiretot_TgC_month_ORCHIDEE-CNP,fFiretot_TgC_month_CLM5.0,fFiretot_TgC_month_ISBA-CTRIP,fFiretot_TgC_month_LPJ,fFiretot_TgC_month_SDGVM,fFiretot_TgC_month_VISIT,mean,std,count,Month,fFiretot_TgC_month_OCN,fFiretot_TgC_month_YIBs,fFiretot_TgC_month_ISAM,Year
0,2008-01-15,17.251555,14.278623,20.144920,0.0,0.0,25.036818,20.253562,25.344799,37.862121,18.005945,17.817834,10.815373,10,1,0.0,0.0,0.0,2008
1,2008-02-15,9.340184,21.739282,20.864381,0.0,0.0,21.552878,13.239303,26.249970,35.419403,16.449389,16.485479,10.636873,10,2,0.0,0.0,0.0,2008
2,2008-03-15,6.528211,32.439310,20.144920,0.0,0.0,13.916302,9.857798,25.344799,37.862121,21.342671,16.743613,12.326827,10,3,0.0,0.0,0.0,2008
3,2008-04-15,7.409542,31.586002,20.144920,0.0,0.0,7.807634,14.991987,25.344799,36.640762,22.742262,16.666791,12.117793,10,4,0.0,0.0,0.0,2008
4,2008-05-15,5.676847,49.469675,20.144920,0.0,0.0,6.314067,19.867079,25.344799,37.862121,25.727183,19.040669,15.595440,10,5,0.0,0.0,0.0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2019-08-15,72.452662,40.737665,20.719732,0.0,0.0,30.532673,100.084512,22.562840,33.529802,24.158951,34.477884,29.354976,10,8,0.0,0.0,0.0,2019
140,2019-09-15,88.235119,50.136619,20.719733,0.0,0.0,63.973931,126.734333,22.562840,32.448196,25.190528,43.000130,38.195677,10,9,0.0,0.0,0.0,2019
141,2019-10-15,47.411970,31.741872,20.719732,0.0,0.0,100.163997,66.004480,22.562840,33.529802,25.064699,34.719939,28.776715,10,10,0.0,0.0,0.0,2019
142,2019-11-15,18.577060,21.131917,20.719733,0.0,0.0,74.819297,23.489458,22.562840,32.448196,22.152184,23.590068,19.642880,10,11,0.0,0.0,0.0,2019
